### This transforms the city column in the survey into an interactive map that can be exported into an html file.
##### As of February 5, 2025

##### import packages

In [3]:

from geopy.geocoders import Nominatim
from geopy.exc import GeocoderNotFound, GeocoderUnavailable
import pandas as pd

In [2]:
def get_coordinates(row):
    try:
        city = row['City_Municipality_or_Province']
        region = row['Province_Region_or_State']
        country = row['Country']
        location = geolocator.geocode(f"{city}, {region}, {country}")
        if location:
            return location.latitude, location.longitude
        else:
            return None, None  # Handle cases where no location is found
    except (GeocoderNotFound, GeocoderUnavailable) as e:
        print(f"Geocoding error for {city}, {region}, {country}: {e}")
        return None, None

##### DimLoc has some rows deleted to avoid out of place markers
##### DimLoc -> output_locations_with_coordinates.csv -> checked_locations_with_coordinates.xlsx

In [ ]:
# Load data from CSV
df = pd.read_csv('DimLoc.csv')

# Initialize Nominatim geocoder
geolocator = Nominatim(user_agent="my_geocoder")

# Apply the function to each row
df[['Latitude', 'Longitude']] = df.apply(get_coordinates, axis=1, result_type='expand')

# Save the results to a new CSV file
df.to_csv('locations_with_coordinates.csv', index=False) # last run was >7 mins

### After manually checking locations_with_coordinates.csv, new file is saved as xlsx
### Blank rows are deleted 

In [4]:
# import the csv for manual checking
dfloc = pd.read_excel('xlsx_locations_with_coordinates.xlsx', sheet_name = 'locations_with_coordinates')


In [5]:
# visually inspect the map
import folium 

# using folium.Map to create a map centered around the Philippines 
m = folium.Map(location=[12.8797, 121.7740], zoom_start=6) 

# marker size
marker_size = 'small'  # Options: 'small', 'medium', 'large'

# using folium.Marker to add markers to the map 
for _, row in dfloc.iterrows(): 
    folium.Marker( 
        location=[row['Latitude'], row['Longitude']], 
        popup=row['Province_Region_or_State'], 
        tooltip=row['City_Municipality_or_Province'] 
        ).add_to(m)     
# using m to display the map 
m

In [6]:
# using m.save to save the map
m.save('location_map_feb5.html')

In [6]:
# visually inspect the map
import folium 

# using folium.Map to create a map centered around the geographical center of the world
m = folium.Map(location=[20.0, 0.0], zoom_start=2)  

# marker size
marker_size = 'small'  # Options: 'small', 'medium', 'large'

# using folium.Marker to add markers to the map 
for _, row in dfloc.iterrows(): 
    folium.Marker( 
        location=[row['Latitude'], row['Longitude']], 
        popup=row['Province_Region_or_State'], 
        tooltip=row['City_Municipality_or_Province'] 
        ).add_to(m)     
# using m to display the map 
m

### Using only the provinces: create a dictionary of the regions and the answered cities in the survey

In [8]:

# initial dictionary of regions and locations
mapping = {
    'Metro Manila': ['Caloocan', 'Las Piñas', 'Mandaluyong', 'Manila', 'Marikina', 'Pateros', 'Pasay', 'Pasig', 
                     'Quezon City', 'San Juan', 'Taguig', 'Valenzuela', 'Malabon', 'Makati', 'Paranaque', 
                     'Muntinlupa City', 'San Jose City', 'Muntinlupa', 'Las Pinas City', 'Bay'],
    'Ilocos Region': ['Ilocos Norte', 'Ilocos Sur', 'La Union', 'Pangasinan'],
    'Cagayan Valley': ['Batanes', 'Cagayan', 'Isabela', 'Nueva Vizcaya', 'Quirino'],
    'Central Luzon': ['Aurora', 'Bataan', 'Bulacan', 'Nueva Ecija', 'Pampanga', 'Tarlac', 'Zambales', 'San Jose del Monte'],
    'North Luzon': ['Baguio', 'Baguio City', 'Benguet', 'San Fernando City', 'San Jose Del Monte', 'Malolos City', 
                    'Mabalacat City', 'Baliwag', 'Baliuag City'],
    'Calabarzon': ['Batangas', 'Cavite', 'Laguna', 'Quezon', 'Rizal', 'Antipolo City', 'Antipolo', 'Calamba City', 
                   'Dasmariñas', 'Imus', 'Lucena City', 'San Pedro', 'Santa Rosa', 'Taytay', 'Bacoor City', 
                   'Lipa City', 'Angono', 'Cabuyao', 'Biñan City'],
    'Mimaropa': ['Marinduque', 'Occidental Mindoro', 'Oriental Mindoro', 'Palawan', 'Romblon', 'Puerto Princesa'],
    'Bicol Region': ['Albay', 'Camarines Norte', 'Camarines Sur', 'Catanduanes', 'Masbate', 'Sorsogon', 'Naga City', 
                     'Daet', 'Bicol'],
    'Western Visayas': ['Aklan', 'Antique', 'Capiz', 'Guimaras', 'Iloilo', 'Negros Occidental', 'Bacolod City', 
                        'Sagay City', 'Panay'],
    'Central Visayas': ['Bohol', 'Cebu', 'Negros Oriental', 'Siquijor', 'Tagbilaran City'],
    'Eastern Visayas': ['Biliran', 'Eastern Samar', 'Leyte', 'Northern Samar', 'Samar', 'Southern Leyte', 'Tacloban City', 
                        'Catbalogan'],
    'Zamboanga Peninsula': ['Zamboanga del Norte', 'Zamboanga del Sur', 'Zamboanga Sibugay', 'Zamboanga City'],
    'Northern Mindanao': ['Bukidnon', 'Camiguin', 'Lanao del Norte', 'Misamis Occidental', 'Misamis Oriental', 
                          'Butuan City', 'Cagayan de Oro City', 'Gingoog City'],
    'Davao Region': ['Davao de Oro', 'Davao del Norte', 'Davao del Sur', 'Davao Occidental', 'Davao Oriental', 'Davao City', 'Davao'],
    'Soccsksargen': ['Cotabato', 'Sarangani', 'South Cotabato', 'Sultan Kudarat', 'General Santos City'],
    'Caraga': ['Agusan del Norte', 'Agusan del Sur', 'Dinagat Islands', 'Surigao del Norte', 'Surigao del Sur', 'Siargao'],
    'Bangsamoro': ['Basilan', 'Lanao del Sur', 'Maguindanao', 'Sulu', 'Tawi-Tawi'],
    'International': ['Sydney', 'Calgary', 'United Kingdom', 'Tallinn', 'Manama', 'Singapore', 'Cairo', 'Ho Chi Minh', 
                      'Melbourne', 'Madrid', 'Alexandria', 'Tallinn']
}


In [21]:
# using m.save to save the map
m.save('location_map_feb5.html')